# Data augmentation

Combines the df_berce_clean CSV and the swiss_datas_v0 CSV and saves it to the `/data` directory

In [11]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkt

In [12]:
df_berce = pd.read_csv('./processed_data/df_berce_clean.csv')
df_swiss = pd.read_csv('./data/swiss_datas_v0.csv')

In [13]:
from pyproj import Transformer
 
transformer_lv95 = Transformer.from_crs("EPSG:4326", "EPSG:2056")
 
# Fonction pour transformer les coordonnées géographiques en EPSG:21781 (LV95)
def add_coordinates_lv95(row):
    # Vérifier la validité des coordonnées
    if pd.notna(row['decimalLatitude']) and pd.notna(row['decimalLongitude']):
        if -90 <= row['decimalLatitude'] <= 90 and -180 <= row['decimalLongitude'] <= 180:
            # Transformation en EPSG:2056 (LV95)
            east_lv95, north_lv95 = transformer_lv95.transform(row['decimalLatitude'], row['decimalLongitude'])
            return pd.Series({
                'east_lv95': east_lv95,
                'north_lv95': north_lv95
            })
    # Retourner NaN si les coordonnées sont invalides
    return pd.Series({
        'east_lv95': float('nan'),
        'north_lv95': float('nan')
    })

# Appliquer la transformation à chaque ligne du dataframe
df_berce[['east_lv95', 'north_lv95']] = df_berce.apply(add_coordinates_lv95, axis=1)

In [14]:
df_berce = df_berce.drop(columns=['id', 'decimalLatitude', 'decimalLongitude'])
df_berce.head()

,elevation,date,east_lv95,north_lv95
0,658.0,1899-12-30 00:00:00,2.636863e+06,1.127713e+06
1,500.0,1905-06-29 00:00:00,2.604850e+06,1.222025e+06
2,500.0,1905-06-28 00:00:00,2.604850e+06,1.222025e+06
3,1448.0,1970-01-01 00:00:00,2.565600e+06,1.138100e+06
4,1246.5,1970-01-01 00:00:00,2.579200e+06,1.133200e+06


In [15]:
df_swiss.rename(columns={
    'idx': 'id',
    'Bodentyp': 'SoilType',
    'WASSERSPEI': 'WaterRetention',
    'NAHRSTOFF': 'NutrientContent',
    'WASSERDURC': 'WaterPermeability'
}, inplace=True)

df_swiss.head()

,id,geometry,SoilType,WaterRetention,NutrientContent,WaterPermeability
0,35,"POLYGON ((2485071.58 1109261.72, 2486071.58 11...","orthic Luvisol; eutric, calcaric Cambisol",4.0,4.0,6.0
1,36,"POLYGON ((2485071.58 1110261.72, 2486071.58 11...","orthic Luvisol; eutric, calcaric Cambisol",4.0,4.0,6.0
2,260,"POLYGON ((2486071.58 1109261.72, 2487071.58 11...","orthic Luvisol; eutric, calcaric Cambisol",4.0,4.0,6.0
3,261,"POLYGON ((2486071.58 1110261.72, 2487071.58 11...","orthic Luvisol; eutric, calcaric Cambisol",4.0,4.0,6.0
4,262,"POLYGON ((2486071.58 1111261.72, 2487071.58 11...","orthic Luvisol; eutric, calcaric Cambisol",4.0,4.0,6.0


In [16]:
gdf_berce = gpd.GeoDataFrame(df_berce, geometry=gpd.points_from_xy(df_berce['east_lv95'], df_berce['north_lv95']))
gdf_swiss = gpd.GeoDataFrame(df_swiss, geometry=gpd.GeoSeries.from_wkt(df_swiss['geometry']))

gdf_berce.crs = 'EPSG:2056' # LV95
gdf_swiss.crs = 'EPSG:2056'

# Perform spatial join
joined_gdf = gpd.sjoin(gdf_swiss, gdf_berce, how='inner', predicate='contains')
joined_gdf = gpd.GeoDataFrame(joined_gdf, geometry='geometry', crs="EPSG:2056")

# Show the result
joined_gdf[500:510]

,id,geometry,SoilType,WaterRetention,NutrientContent,WaterPermeability,index_right,elevation,date,east_lv95,north_lv95
4377,17209,"POLYGON ((2561071.58 1183261.72, 2562071.58 11...","eutric, gleyic, dystric, calcaric Cambisol",5.0,5.0,5.0,16935,452.5,2016-08-12 00:00:00,2.561209e+06,1.183628e+06
4398,17230,"POLYGON ((2561071.58 1204261.72, 2562071.58 12...",-,-9999.0,-9999.0,-9999.0,18096,469.5,2017-07-22 00:00:00,2.561472e+06,1.204964e+06
4413,17245,"POLYGON ((2561071.58 1219261.72, 2562071.58 12...",Eutric Regosol; Rendzina (Jura); Ranker; Lithosol,3.0,3.0,5.0,19681,848.5,2018-06-15 00:00:00,2.561176e+06,1.219265e+06
4413,17245,"POLYGON ((2561071.58 1219261.72, 2562071.58 12...",Eutric Regosol; Rendzina (Jura); Ranker; Lithosol,3.0,3.0,5.0,21150,NaN,2020-05-26 19:02:00,2.561180e+06,1.219267e+06
4413,17245,"POLYGON ((2561071.58 1219261.72, 2562071.58 12...",Eutric Regosol; Rendzina (Jura); Ranker; Lithosol,3.0,3.0,5.0,19807,848.0,2018-06-15 00:00:00,2.561191e+06,1.219284e+06
4413,17245,"POLYGON ((2561071.58 1219261.72, 2562071.58 12...",Eutric Regosol; Rendzina (Jura); Ranker; Lithosol,3.0,3.0,5.0,19416,850.0,2018-06-15 00:00:00,2.561276e+06,1.219353e+06
4413,17245,"POLYGON ((2561071.58 1219261.72, 2562071.58 12...",Eutric Regosol; Rendzina (Jura); Ranker; Lithosol,3.0,3.0,5.0,18840,NaN,2018-08-21 17:52:29+00:00,2.561286e+06,1.219360e+06
4413,17245,"POLYGON ((2561071.58 1219261.72, 2562071.58 12...",Eutric Regosol; Rendzina (Jura); Ranker; Lithosol,3.0,3.0,5.0,22651,NaN,2021-06-04 18:24:59,2.561287e+06,1.219363e+06
4413,17245,"POLYGON ((2561071.58 1219261.72, 2562071.58 12...",Eutric Regosol; Rendzina (Jura); Ranker; Lithosol,3.0,3.0,5.0,18993,896.0,2018-04-28 00:00:00,2.561110e+06,1.219515e+06
4413,17245,"POLYGON ((2561071.58 1219261.72, 2562071.58 12...",Eutric Regosol; Rendzina (Jura); Ranker; Lithosol,3.0,3.0,5.0,19297,896.0,2018-04-28 00:00:00,2.561110e+06,1.219515e+06


In [19]:
# Writes data into a csv
joined_gdf.to_csv("data/augmented_data.csv", index=False)
